In [20]:
import pandas as pd
pd.options.display.float_format = '{:,.4f}'.format
pd.options.display.max_rows = 100
pd.options.display.max_columns = 40
import numpy as np
import os,random, math
from tqdm import tqdm
from copy import deepcopy
from collections import Counter

# Visualization
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import font_manager, rc
plt.rcParams['font.family'] = 'NanumGothic'
import platform
if platform.system() == 'Windows':
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)
else:    
    rc('font', family='AppleGothic')

matplotlib.rcParams['axes.unicode_minus'] = False

# Warnings
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

import datetime
import re

from shapely.geometry import Point, shape, mapping
from shapely.ops import unary_union


#from geoband.API import *
import folium
from folium import plugins
import json
import geopandas as gpd
from geopandas import gpd
from haversine import haversine

In [21]:
directory = '/Users/seungji/Desktop/STDA_finalproject/STDA/'

In [22]:
month = ['01','02','03','04','05','06','07','08','09','10']

In [23]:
for i in month:
    exec(f"bus{i} = pd.read_csv(directory + 'data/bus/BUS_STATION_BOARDING_MONTH_2021{i}.csv',encoding = 'cp949')")

In [24]:
merge = pd.read_csv('/Users/seungji/Library/Mobile Documents/com~apple~CloudDocs/STDA/Project/data/merge2.csv')

In [25]:
location = pd.read_csv('/Users/seungji/Library/Mobile Documents/com~apple~CloudDocs/STDA/Project/data/sdot_censor_location.csv')
# 경기도 과천시 데이터 제거 
location = location.iloc[2:].reset_index(drop=True)
location = location[location.사이트명.isin(merge.시리얼.unique())].reset_index(drop=True)

# 300m 거리에 있는 버스정류장 뽑기 

In [26]:
bus_g = pd.read_csv('/Users/seungji/Library/Mobile Documents/com~apple~CloudDocs/STDA/Project/data/서울시버스정류소좌표데이터(2021.01.14.).csv',encoding = 'cp949')

In [27]:
bb = list()
for b in bus_g['ARS-ID']:
    if len(str(b))<5: 
        bb.append('0'+str(b) )
    else: bb.append(str(b))
bus_g['ARS-ID'] = bb

In [28]:
location['coord'] = list(zip(location.위도,location.경도))
bus_g['coord'] = list(zip(bus_g.Y좌표,bus_g.X좌표))

In [29]:
bus_dic = dict()
for i, c in enumerate(location.coord):
    temp_busnm = []
    for j, s in enumerate(bus_g.coord):
        if haversine(c, s) <= 0.306:
            temp_busnm.append(bus_g['ARS-ID'][j])
    bus_dic[location.사이트명[i]] = [temp_busnm,['<300']]

In [30]:
bus_dic

{4019: [['05228', '05263'], ['<300']],
 4040: [['01518',
   '01529',
   '01547',
   '01548',
   '01556',
   '01792',
   '01800',
   '01807',
   '01839',
   '01845'],
  ['<300']],
 4043: [['02001', '02140', '02141', '02142', '02156', '02253', '02925'],
  ['<300']],
 4005: [['23101', '23102', '23104', '23913'], ['<300']],
 4007: [['23158', '23159', '23160', '23181', '23909'], ['<300']],
 4032: [['04179', '04180', '04867'], ['<300']],
 4035: [['03173',
   '03174',
   '03175',
   '03500',
   '03505',
   '03509',
   '03583',
   '03585',
   '03753',
   '03754'],
  ['<300']],
 4001: [['22011',
   '22012',
   '22172',
   '22173',
   '22406',
   '22407',
   '22408',
   '22409',
   '22410',
   '22411',
   '22412',
   '22413',
   '22838',
   '22850',
   '22885',
   '23285',
   '23286',
   '23574',
   '23580',
   '23641',
   '23900'],
  ['<300']],
 4002: [['23103', '23105', '23106', '23429', '23430', '23910'], ['<300']],
 4003: [['23218',
   '23241',
   '23254',
   '23257',
   '23258',
   '23259',

In [31]:
for c, sub in zip(bus_dic.keys(), bus_dic.values()):
    if sub[0] == []:
        print(c)
        print('hi')          

## 데이터 결합

In [38]:
for i in month:
    exec(f"bus{i} = bus{i}[bus{i}.버스정류장ARS번호.isin(bus_g['ARS-ID'].unique())].reset_index(drop=True)")
    exec(f"bus{i} = bus{i}.groupby(['사용일자','버스정류장ARS번호']).sum()[['승차총승객수','하차총승객수']].reset_index()")

In [43]:
bus_all = pd.DataFrame()
for i in month:
    exec(f"bus_all = pd.concat([bus_all, bus{i}], axis = 0)")

In [44]:
bus_all.shape

(3209629, 4)

In [46]:
bus_all.columns

Index(['사용일자', '버스정류장ARS번호', '승차총승객수', '하차총승객수'], dtype='object')

In [47]:
bus_mrg = pd.DataFrame()
for c in merge.시리얼.unique():
    tmp = bus_all.loc[bus_all.버스정류장ARS번호.isin(bus_dic[c][0]),['사용일자','버스정류장ARS번호','승차총승객수','하차총승객수']].groupby(['사용일자']).mean().reset_index()
    tmp['시리얼'] = c
    tmp['총승객수'] = (tmp.승차총승객수 + tmp.하차총승객수)/2
    bus_mrg = pd.concat([bus_mrg,tmp],axis = 0)
    

In [49]:
bus_mrg = bus_mrg[['시리얼','사용일자','총승객수']]

In [51]:
bus_mrg.사용일자 = list(map(lambda x: str(x)[:4]+'-'+str(x)[4:6]+'-'+str(x)[6:8],bus_mrg.사용일자))

In [56]:
merge = pd.merge(merge, bus_mrg, left_on = ['시리얼','date'], right_on = ['시리얼','사용일자'], how = 'left')

In [57]:
merge

,시리얼,date,방문자수,총승객수_x,사용일자,총승객수_y
0,"2,992.0000",2021-01-04,"6,594.7167","15,223.5000",2021-01-04,"1,248.3000"
1,"2,992.0000",2021-01-05,"6,290.3333","14,932.0000",2021-01-05,"1,239.3000"
2,"2,992.0000",2021-01-06,"6,222.3667","15,729.0000",2021-01-06,"1,254.0000"
3,"2,992.0000",2021-01-07,"5,919.0333","16,100.5000",2021-01-07,"1,319.9000"
4,"2,992.0000",2021-01-08,"5,982.3000","14,750.0000",2021-01-08,"1,207.8000"
...,...,...,...,...,...,...
26314,"4,050.0000",2021-10-19,69.1000,"7,098.0000",2021-10-19,357.6875
26315,"4,050.0000",2021-10-20,71.4667,"9,299.0000",2021-10-20,411.1875
26316,"4,050.0000",2021-10-21,67.8667,"7,461.0000",2021-10-21,357.6250
26317,"4,050.0000",2021-10-22,67.0000,"7,634.0000",2021-10-22,371.1250


In [58]:
merge = merge.drop(columns = ['사용일자'])

In [60]:
merge = merge.rename(columns = {'총승객수_x':'지하철승객','총승객수_y':'버스승객'})

In [61]:
directory = '/Users/seungji/Library/Mobile Documents/com~apple~CloudDocs/STDA/Project/'

In [62]:
merge.to_csv(directory +'data/merge3.csv',index = False )